In [119]:
import torch
import numpy as np
import meshio

from torchmcubes import marching_cubes, grid_interp

import distillation

In [120]:
nsr = torch.load("nsr000.pt")

C:\Users\andrewhc\AppData\Local\Temp\ipykernel_13436\633730493.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nsr = torch.load("nsr000.pt")


In [121]:
fg_sdf = nsr["s.fg_sdf"][0][0]

In [122]:
verts, faces = marching_cubes(fg_sdf, thresh=0.0)

# removes duplicate vertices
verts, reverse_ind = torch.unique(verts, dim=0, return_inverse=True)
faces = reverse_ind[faces]

In [123]:
mesh = meshio.Mesh(
    verts.detach().cpu().numpy().tolist(),
    [("triangle", faces.detach().cpu().numpy().tolist())]  # + albedo (color) & roughness
)
mesh.write("calculator.obj")

In [124]:
normal = torch.zeros(verts.shape)
albedo = torch.zeros(verts.shape[0])
roughness = torch.zeros(verts.shape[0])

In [128]:
# Fx(A,B,C) -> Fx(A,B,C)x(X,Y,Z)
vf = verts[faces]
face_norm = torch.linalg.cross(vf[:,1]-vf[:,0], vf[:,2]-vf[:,0])

In [129]:
face_norm

tensor([[-0.0019, -0.0049, -0.0038],
        [-0.0984, -0.3550, -0.3149],
        [-0.0115, -0.0491, -0.0377],
        ...,
        [ 0.0103, -0.0044,  0.0767],
        [-0.0040,  0.0048,  0.0321],
        [ 0.0040,  0.0044,  0.0295]], device='cuda:0')

In [64]:
faces.clone().float()

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00],
        [3.0000e+00, 4.0000e+00, 5.0000e+00],
        [6.0000e+00, 7.0000e+00, 8.0000e+00],
        ...,
        [2.1699e+04, 2.1700e+04, 2.1701e+04],
        [2.1702e+04, 2.1703e+04, 2.1704e+04],
        [2.1705e+04, 2.1706e+04, 2.1707e+04]], device='cuda:0')

In [49]:
faces[0] = face_norm[0,np.newaxis]
faces[0]

tensor([0, 0, 0], device='cuda:0', dtype=torch.int32)

In [53]:
face_norm[1,np.newaxis]

tensor([[-0.0984, -0.3550, -0.3149]], device='cuda:0')

In [72]:
ftest = faces.clone().float()
ftest[0] = face_norm[0]
ftest[0]

tensor([-0.0019, -0.0049, -0.0038], device='cuda:0')

In [71]:
face_norm[2]

tensor([-0.0115, -0.0491, -0.0377], device='cuda:0')

In [78]:
ftest

tensor([[-1.8538e-03, -4.8872e-03, -3.7509e-03],
        [ 3.0000e+00,  4.0000e+00,  5.0000e+00],
        [ 6.0000e+00,  7.0000e+00,  8.0000e+00],
        ...,
        [ 2.1699e+04,  2.1700e+04,  2.1701e+04],
        [ 2.1702e+04,  2.1703e+04,  2.1704e+04],
        [ 2.1705e+04,  2.1706e+04,  2.1707e+04]], device='cuda:0')

In [189]:
# for norm, (a,b,c) in zip(face_norm, faces):
#     verts[[a,b,c]] += norm
# verts[faces] = face_norm
# vert_norm = np.zeros(verts.shape)
# np.add.at(vert_norm, faces, face_norm)

# supports simulataneous summation for multiple indices
vert_norm = torch.zeros(verts.shape, device="cuda")
vert_norm.index_put_(indices=(faces.flatten(),), values=torch.repeat_interleave(face_norm, 3, dim=0), accumulate=True)
vert_norm

# assuming no duplicate indices
vert_norm = torch.zeros(verts.shape, device="cuda")
vert_norm[faces.flatten()] = torch.repeat_interleave(face_norm, 3, dim=0)
vert_norm

tensor([[-1.0000,  0.0659, -0.1497],
        [-1.0000,  0.2283,  0.3286],
        [-0.8532,  0.5779, -0.0832],
        ...,
        [ 0.4127,  0.5136, -0.1440],
        [ 0.3991,  0.2321,  0.4544],
        [ 1.0000, -0.3275,  0.1101]], device='cuda:0')

In [188]:
faces, faces.flatten(), face_norm, torch.repeat_interleave(face_norm, 3, dim=0)

(tensor([[ 887,  845,  888],
         [ 887, 1079, 1077],
         [ 888, 1079,  887],
         ...,
         [2481, 2463, 2460],
         [2359, 2463, 2464],
         [2481, 2464, 2463]], device='cuda:0'),
 tensor([ 887,  845,  888,  ..., 2481, 2464, 2463], device='cuda:0'),
 tensor([[-0.0019, -0.0049, -0.0038],
         [-0.0984, -0.3550, -0.3149],
         [-0.0115, -0.0491, -0.0377],
         ...,
         [ 0.0103, -0.0044,  0.0767],
         [-0.0040,  0.0048,  0.0321],
         [ 0.0040,  0.0044,  0.0295]], device='cuda:0'),
 tensor([[-0.0019, -0.0049, -0.0038],
         [-0.0019, -0.0049, -0.0038],
         [-0.0019, -0.0049, -0.0038],
         ...,
         [ 0.0040,  0.0044,  0.0295],
         [ 0.0040,  0.0044,  0.0295],
         [ 0.0040,  0.0044,  0.0295]], device='cuda:0'))

In [180]:
faces[0], faces[1], faces[2], face_norm[0], face_norm[1], face_norm[2]

(tensor([887, 845, 888], device='cuda:0'),
 tensor([ 887, 1079, 1077], device='cuda:0'),
 tensor([ 888, 1079,  887], device='cuda:0'),
 tensor([-0.0019, -0.0049, -0.0038], device='cuda:0'),
 tensor([-0.0984, -0.3550, -0.3149], device='cuda:0'),
 tensor([-0.0115, -0.0491, -0.0377], device='cuda:0'))

In [176]:
vert_norm[887]

tensor([-0.2772, -1.0000, -0.6548], device='cuda:0')

In [179]:
vert_norm, faces[0:40], face_norm

(tensor([[-1.0000,  0.0659, -0.1497],
         [-1.0000,  0.2283,  0.3286],
         [-0.8532,  0.5779, -0.0832],
         ...,
         [ 0.4127,  0.5136, -0.1440],
         [ 0.3991,  0.2321,  0.4544],
         [ 1.0000, -0.3275,  0.1101]], device='cuda:0'),
 tensor([[ 887,  845,  888],
         [ 887, 1079, 1077],
         [ 888, 1079,  887],
         [1077, 1250, 1247],
         [1079, 1250, 1077],
         [1247, 1419, 1417],
         [1250, 1419, 1247],
         [1419, 1489, 1417],
         [ 748,  701,  750],
         [ 748,  888,  845],
         [ 748,  750,  888],
         [ 750,  891,  888],
         [1079,  888, 1084],
         [ 888,  891, 1084],
         [1250, 1079, 1254],
         [1079, 1083, 1254],
         [1419, 1250, 1422],
         [1250, 1254, 1422],
         [1422, 1489, 1419],
         [1517, 1489, 1422],
         [ 750,  687,  753],
         [ 701,  687,  750],
         [ 891,  750,  894],
         [ 750,  753,  894],
         [1084,  891, 1087],
         [ 891

In [165]:
vert_norm.shape, faces.shape, face_norm.shape

(torch.Size([3643, 1, 3]), torch.Size([7236, 3]), torch.Size([7236, 3]))

In [86]:
verts[[0,1,2]] #+ torch.tensor([1,2,1], device="cuda")

tensor([[23.0000, 31.9623,  9.0000],
        [22.9006, 32.0000,  9.0000],
        [23.0000, 32.0000,  8.9509]], device='cuda:0')

In [96]:
print(verts[[0,1,2]])
verts[[0,1,2,0,1,2]] += torch.tensor([100,2,1], device="cuda")
print(verts[[0,1,2]])

tensor([[225.0000,  39.9623,  13.0000],
        [224.9006,  40.0000,  13.0000],
        [225.0000,  40.0000,  12.9509]], device='cuda:0')
tensor([[325.0000,  41.9623,  14.0000],
        [324.9006,  42.0000,  14.0000],
        [325.0000,  42.0000,  13.9509]], device='cuda:0')


In [100]:
verts, faces

(tensor([[325.0000,  41.9623,  14.0000],
         [324.9006,  42.0000,  14.0000],
         [325.0000,  42.0000,  13.9509],
         ...,
         [ 32.1816,  33.0000,  55.0000],
         [ 32.0000,  33.1627,  55.0000],
         [ 32.0000,  33.0000,  55.0243]], device='cuda:0'),
 tensor([[    0,     0,     0],
         [    3,     4,     5],
         [    6,     7,     8],
         ...,
         [21699, 21700, 21701],
         [21702, 21703, 21704],
         [21705, 21706, 21707]], device='cuda:0', dtype=torch.int32))

In [106]:
faces.shape[0]*faces.shape[1], torch.unique(faces).shape  # literally unique (there will be no duplicated elements)

(21708, torch.Size([21706]))

In [127]:
verts.shape, #torch.unique(verts, dim=0).shape

(torch.Size([3643, 3]),)

In [113]:
verts

tensor([[325.0000,  41.9623,  14.0000],
        [324.9006,  42.0000,  14.0000],
        [325.0000,  42.0000,  13.9509],
        ...,
        [ 32.1816,  33.0000,  55.0000],
        [ 32.0000,  33.1627,  55.0000],
        [ 32.0000,  33.0000,  55.0243]], device='cuda:0')

In [115]:
compact_vert, reverse_ind = torch.unique(verts, dim=0, return_inverse=True)
reverse_ind

tensor([3644, 3643, 3645,  ..., 2481, 2464, 2463], device='cuda:0')

In [117]:
faces

tensor([[    0,     0,     0],
        [    3,     4,     5],
        [    6,     7,     8],
        ...,
        [21699, 21700, 21701],
        [21702, 21703, 21704],
        [21705, 21706, 21707]], device='cuda:0', dtype=torch.int32)

In [118]:
new_faces = reverse_ind[faces]

tensor([[3644, 3644, 3644],
        [ 887, 1079, 1077],
        [ 888, 1079,  887],
        ...,
        [2481, 2463, 2460],
        [2359, 2463, 2464],
        [2481, 2464, 2463]], device='cuda:0')